# Creación de Esquemas y Tablas - Capa Bronce

Esta notebook crea las **estructuras de datos base** (esquemas y tablas) en la **capa Bronce** del Data Lakehouse usando Databricks.

## ¿Qué hace esta notebook?

1. **Configura parámetros dinámicos** mediante widgets para nombres de esquemas y catálogos
2. **Define y ejecuta DDL** para crear 5 tablas que existen en el transaccional de DataVision:
   - `accounts`: Información básica de cuentas de usuario
   - `account_premium_features`: Features premium adquiridos por cuenta
   - `accounts_subscription`: Suscripciones activas por cuenta
   - `premium_features`: Catálogo de features premium disponibles
   - `subscriptions`: Tipos de suscripción disponibles

---

## Configuración con widgets

### ¿Qué son los widgets en Databricks?

Los **widgets** son controles interactivos que permiten parametrizar notebooks. Se usan para evitar valores hardcodeados y facilitar la reutilización del código.

Beneficios:
- Configurables sin tocar el código
- Útiles para pruebas y entornos educativos
- Permiten ejecutar la misma notebook con distintos valores
- Compatibles con jobs programados

### Widgets usados en esta notebook

- **`alumno`**: Nombre del alumno (para nombres de tablas)
- **`catalogo`**: Catálogo de destino en Databricks 


In [0]:
# Configuración de widgets para parámetros
# Los widgets permiten cambiar estos valores sin modificar el código

# Widget para el nombre del alumno (usado en nombres de esquema)
dbutils.widgets.text("alumno", "bruno", "Nombre del Alumno")

# Widget para el catálogo de destino en Databricks
dbutils.widgets.text("catalogo", "bronce_dev", "Catálogo Destino")

ALUMNO = dbutils.widgets.get("alumno")
CATALOGO = dbutils.widgets.get("catalogo")

## Definición y ejecución de DDL (Data Definition Language)

La siguiente celda contiene la **lógica principal** de creación de tablas. Esta celda:

### ¿Qué hace?

1. **Define las estructuras de tabla** como strings SQL usando f-strings para incluir los parámetros dinámicos (`CATALOGO` y `ALUMNO`)
2. **Crea 5 tablas relacionadas** con un sistema de gestión de cuentas de usuario
3. **Ejecuta los DDL** mediante un bucle que itera sobre cada definición de tabla
4. **Usa `CREATE TABLE IF NOT EXISTS`** para evitar errores si las tablas ya existen

In [0]:
# Definir los DDL como tuplas
ddls = (
    f"""
    CREATE OR REPLACE TABLE {CATALOGO}.datavision_{ALUMNO}.accounts (
        account_id STRING,
        account_name STRING,
        email STRING,
        created_at STRING,
        updated_at STRING,
        fecha_extraccion DATE
    )
    """,
    f"""
    CREATE OR REPLACE TABLE {CATALOGO}.datavision_{ALUMNO}.account_premium_features (
        account_feature_id STRING,
        account_id STRING,
        feature_id STRING,
        purchase_date STRING,
        amount_paid STRING,
        fecha_carga DATE,
        _rescued_data STRING
    )
    """,
    f"""
    CREATE OR REPLACE TABLE  {CATALOGO}.datavision_{ALUMNO}.accounts_subscription (
        account_subscription_id STRING,
        account_id STRING,
        subscription_id STRING,
        start_date STRING,
        end_date STRING,
        fecha_extraccion DATE
    )
    """,
    f"""
    CREATE OR REPLACE TABLE {CATALOGO}.datavision_{ALUMNO}.premium_features (
        feature_id STRING,
        feature_name STRING,
        description STRING,
        base_price STRING,
        created_at STRING,
        updated_at STRING,
        fecha_extraccion DATE
    )
    """,
    f"""
    CREATE OR REPLACE TABLE {CATALOGO}.datavision_{ALUMNO}.subscriptions (
        subscription_id STRING,
        subscription_name STRING,
        max_contents_per_month STRING,
        created_at STRING,
        updated_at STRING,
        fecha_extraccion DATE
    )
    """,
)

# Ejecutar los DDL
for ddl in ddls:
    spark.sql(ddl)

## Mejores prácticas en la capa Bronce

### 1. Campos como STRING por diseño

En la **capa Bronze** (capa raw), todos los campos se definen como `STRING` **intencionalmente**:

**¿Por qué?**
- **Tolerancia a datos sucios**: Evita errores de parsing cuando llegan datos inesperados
- **Flexibilidad máxima**: Permite cargar cualquier tipo de dato sin validaciones estrictas
- **Schema evolution**: Facilita agregar nuevas columnas o cambiar tipos de datos
- **Debugging simplificado**: Los datos "raros" no rompen el proceso de carga

**Transformación posterior**: En capas superiores (plata/oro) se harán las conversiones y validaciones apropiadas.

### 2. Versionado y migraciones de Esquema
 
**¿Qué es una migración de esquema?**

Una **migración de esquema** es el proceso de modificar estructuralmente la base de datos (por ejemplo, agregar/quitar tablas, columnas, tipos de datos, constraints, etc.) de manera controlada y versionada. Las migraciones permiten que los cambios en el esquema estén documentados en archivos de código, puedan replicarse fácilmente en diferentes entornos y se integren con el flujo de desarrollo (CI/CD).

Por ejemplo: Si un día necesitás agregar una columna nueva o cambiar el tipo de dato de un campo, en vez de hacerlo manualmente, creás un archivo de migración donde definís ese cambio y el sistema de migraciones aplica ese script en cada ambiente, evitando inconsistencias.

**Nota:** En este curso **no vamos a ver a fondo herramientas ni técnicas de migraciones de esquema**, pero te lo mencionamos porque en proyectos reales es una práctica fundamental para asegurar el orden y la trazabilidad de los cambios estructurales en la base de datos.

**Herramientas populares para migración y versionado de esquemas:**
 
- **[Flyway](https://www.red-gate.com/products/flyway/community/)**: Una herramienta de migración de esquemas open-source que soporta una amplia variedad de sistemas de bases de datos. Permite definir migraciones con SQL o Java, y se puede integrar fácilmente en procesos de build y despliegue.

- **[Liquibase](https://github.com/liquibase/liquibase-databricks/tree/main)**: Otra herramienta open-source para rastrear, gestionar y aplicar cambios en el esquema de la base de datos. Soporta múltiples formatos para definir los cambios: XML, YAML, JSON y SQL.
 
Ambas herramientas también ofrecen versiones Enterprise/Pro con funcionalidades avanzadas como detección de desvíos ("drift detection"), control de cambios por políticas/reglas, aplicaciones con interfaz gráfica, soporte premium y mucho más.

- **[Guía oficial Databricks](https://community.databricks.com/t5/technical-blog/databricks-schema-versioning-with-flyway-and-liquibase-a-step-by/ba-p/90504)** 

**Beneficios de las migraciones SQL:**
- **Versionado del esquema**: Cada cambio queda registrado en código
- **Rollback seguro**: Posibilidad de revertir cambios problemáticos
- **Entornos consistentes**: Dev, staging y prod tienen el mismo esquema
- **Auditoría**: Historial completo de cambios en la estructura de datos
- **Colaboración**: Los cambios de esquema siguen el flujo de CI/CD